In [1]:
import pyexasol
import pandas as pd
import boto3
import time
import io
import numpy as np
from matplotlib import pyplot as plt


dwh_host = '***'
dwh_port = '8563'
dwh_user = '***'
dwh_pass = '***'

#connect to db
dwh_conn = pyexasol.connect(dsn=dwh_host + ':' + dwh_port, user=dwh_user, password=dwh_pass)

## data function to get iterator data

In [5]:
#get iterator month data
def get_feature_types():

    #get iterator datam
    
    v_sql = """
SELECT distinct
        feature_type
from
    betting_mart.ml_poisson_features
where
    full_time_home_goals is not null
    and home_shots_for_ema is not null
    and away_shots_against_ema is not null
    """
    
    return dwh_conn.export_to_pandas(v_sql)


#get iterator month data
def get_iterator_data(feature_type_filter):

    #get iterator datam
    
    v_sql = """SELECT distinct
        to_char(match_date,'yyyymm') iterator
    FROM 
        betting_mart.ml_poisson_features
    WHERE
        --2014_15 season dropped for stable features
        season <> '2014_2015'
        --2015_16 is first training season
        AND season <> '2015_2016' and
        --feature type filter
        feature_type = '""" + str(feature_type_filter) + """' and
        --ohne null values
        home_goals_for_ema is not null and
        home_goals_against_ema is not null and
        away_goals_for_ema is not null and
        away_goals_against_ema is not null and
        home_xg_for_ema is not null and
        home_xg_against_ema is not null and
        away_xg_for_ema is not null and
        away_xg_against_ema is not null and
        home_shots_for_ema is not null and
        home_shots_against_ema is not null and
        away_shots_for_ema is not null and
        away_shots_against_ema is not null and
        home_sot_for_ema is not null and
        home_sot_against_ema is not null and
        away_sot_for_ema is not null and
        away_sot_against_ema is not null and
        home_corners_for_ema is not null and
        home_corners_against_ema is not null and
        away_corners_for_ema is not null and
        away_corners_against_ema is not null and
        home_deep_for_ema is not null and
        home_deep_against_ema is not null and
        away_deep_for_ema is not null and
        away_deep_against_ema is not null and
        home_ppda_for_ema is not null and
        home_ppda_against_ema is not null and
        away_ppda_for_ema is not null and
        away_ppda_against_ema is not null and
        full_time_home_goals is not null and
        full_time_away_goals is not null
    ORDER BY 1"""

    return dwh_conn.export_to_pandas(v_sql)


def get_training_data(feature_type_filter, month_filter):

    v_sql = """
select
    round(home_goals_for_ema,1)            home_goals_for_ema,
    round(home_goals_against_ema,1)        home_goals_against_ema,
    round(away_goals_for_ema,1)            away_goals_for_ema,
    round(away_goals_against_ema,1)        away_goals_against_ema,
    round(home_xg_for_ema,1)               home_xg_for_ema,
    round(home_xg_against_ema,1)           home_xg_against_ema,
    round(away_xg_for_ema,1)               away_xg_for_ema,
    round(away_xg_against_ema,1)           away_xg_against_ema,
    round(home_shots_for_ema,1)               home_shots_for_ema,
    round(home_shots_against_ema,1)           home_shots_against_ema,
    round(away_shots_for_ema,1)               away_shots_for_ema,
    round(away_shots_against_ema,1)           away_shots_against_ema,
    round(home_sot_for_ema,1)               home_sot_for_ema,
    round(home_sot_against_ema,1)           home_sot_against_ema,
    round(away_sot_for_ema,1)               away_sot_for_ema,
    round(away_sot_against_ema,1)           away_sot_against_ema,
    round(home_corners_for_ema,1)               home_corners_for_ema,
    round(home_corners_against_ema,1)           home_corners_against_ema,
    round(away_corners_for_ema,1)               away_corners_for_ema,
    round(away_corners_against_ema,1)           away_corners_against_ema,
    round(home_deep_for_ema,1)               home_deep_for_ema,
    round(home_deep_against_ema,1)           home_deep_against_ema,
    round(away_deep_for_ema,1)               away_deep_for_ema,
    round(away_deep_against_ema,1)           away_deep_against_ema,
    round(home_ppda_for_ema,1)               home_ppda_for_ema,
    round(home_ppda_against_ema,1)           home_ppda_against_ema,
    round(away_ppda_for_ema,1)               away_ppda_for_ema,
    round(away_ppda_against_ema,1)           away_ppda_against_ema,
    full_time_home_goals               full_time_home_goals,
    full_time_away_goals               full_time_away_goals
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') < """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and
    home_goals_for_ema is not null and
    home_goals_against_ema is not null and
    away_goals_for_ema is not null and
    away_goals_against_ema is not null and
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null and
    full_time_home_goals is not null and
    full_time_away_goals is not null
    """
    
    
    return dwh_conn.export_to_pandas(v_sql)



def get_prediction_data(feature_type_filter, month_filter):
    
    v_sql = """
select
    football_match_his_lid,
    round(home_goals_for_ema,1)            home_goals_for_ema,
    round(home_goals_against_ema,1)        home_goals_against_ema,
    round(away_goals_for_ema,1)            away_goals_for_ema,
    round(away_goals_against_ema,1)        away_goals_against_ema,
    round(home_xg_for_ema,1)               home_xg_for_ema,
    round(home_xg_against_ema,1)           home_xg_against_ema,
    round(away_xg_for_ema,1)               away_xg_for_ema,
    round(away_xg_against_ema,1)           away_xg_against_ema,
    round(home_shots_for_ema,1)               home_shots_for_ema,
    round(home_shots_against_ema,1)           home_shots_against_ema,
    round(away_shots_for_ema,1)               away_shots_for_ema,
    round(away_shots_against_ema,1)           away_shots_against_ema,
    round(home_sot_for_ema,1)               home_sot_for_ema,
    round(home_sot_against_ema,1)           home_sot_against_ema,
    round(away_sot_for_ema,1)               away_sot_for_ema,
    round(away_sot_against_ema,1)           away_sot_against_ema,
    round(home_corners_for_ema,1)               home_corners_for_ema,
    round(home_corners_against_ema,1)           home_corners_against_ema,
    round(away_corners_for_ema,1)               away_corners_for_ema,
    round(away_corners_against_ema,1)           away_corners_against_ema,
    round(home_deep_for_ema,1)               home_deep_for_ema,
    round(home_deep_against_ema,1)           home_deep_against_ema,
    round(away_deep_for_ema,1)               away_deep_for_ema,
    round(away_deep_against_ema,1)           away_deep_against_ema,
    round(home_ppda_for_ema,1)               home_ppda_for_ema,
    round(home_ppda_against_ema,1)           home_ppda_against_ema,
    round(away_ppda_for_ema,1)               away_ppda_for_ema,
    round(away_ppda_against_ema,1)           away_ppda_against_ema
from
    betting_mart.ml_poisson_features
where
    --filter for iterator
    to_char(match_date,'yyyymm') = """ + str(month_filter) + """ and
    feature_type = '""" + str(feature_type_filter) + """' and
    home_goals_for_ema is not null and
    home_goals_against_ema is not null and
    away_goals_for_ema is not null and
    away_goals_against_ema is not null and
    home_xg_for_ema is not null and
    home_xg_against_ema is not null and
    away_xg_for_ema is not null and
    away_xg_against_ema is not null and
    home_shots_for_ema is not null and
    home_shots_against_ema is not null and
    away_shots_for_ema is not null and
    away_shots_against_ema is not null and
    home_sot_for_ema is not null and
    home_sot_against_ema is not null and
    away_sot_for_ema is not null and
    away_sot_against_ema is not null and
    home_corners_for_ema is not null and
    home_corners_against_ema is not null and
    away_corners_for_ema is not null and
    away_corners_against_ema is not null and
    home_deep_for_ema is not null and
    home_deep_against_ema is not null and
    away_deep_for_ema is not null and
    away_deep_against_ema is not null and
    home_ppda_for_ema is not null and
    home_ppda_against_ema is not null and
    away_ppda_for_ema is not null and
    away_ppda_against_ema is not null
    """
    
    return dwh_conn.export_to_pandas(v_sql)

## data function to get home and away data

In [6]:

import pandas
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor


#truncate result table
#dwh_conn.execute('truncate table BETTING_DV.football_match_his_l_s_ml_poisson_expected_goals')


#get different feature types
df_feature_iterator = get_feature_types()

for index, row in df_feature_iterator.iterrows():
    print('Loop for feature type: ' + str(row['FEATURE_TYPE']))
    
    print('...getting data monthly loops for simulation')
    
    df_month_iterator = get_iterator_data(row['FEATURE_TYPE'])
    
    for index2, row2 in df_month_iterator.iterrows():
        
        print('...simulation for month ' + str(row2['ITERATOR']))
        
        print('.....getting training data')
        
        #get data to train home expGoals model
        df_train_data = get_training_data(row['FEATURE_TYPE'], row2['ITERATOR'])
    
        #split features and pred classes
        df_train_x = df_train_data.iloc[:,:28]
        df_train_home_y = df_train_data.iloc[:,28]
        df_train_away_y = df_train_data.iloc[:,29]
                
        print('......training models')
        
        print('......train_x: ' + str(df_train_x.shape))
        print('......train home y: ' + str(df_train_home_y.shape))
        print('......train away y: ' + str(df_train_away_y.shape))
        
        
        model_home = RandomForestRegressor(
                                            max_depth = 15,
                                            min_samples_leaf = 10,
                                            min_samples_split = 150,
                                            max_leaf_nodes = 100,
                                            n_estimators = 600,
                                            max_features = 10,
                                            random_state=0
                                            ).fit(df_train_x, df_train_home_y)
        
        
        model_away = RandomForestRegressor(
                                            max_depth = 15,
                                            min_samples_leaf = 10,
                                            min_samples_split = 150,
                                            max_leaf_nodes = 100,
                                            n_estimators = 500,
                                            max_features = 10,
                                            random_state=0
                                            ).fit(df_train_x, df_train_away_y)


        print('......getting prediction data')
        
        #get data to train home expGoals model
        df_pred_data = get_prediction_data(row['FEATURE_TYPE'], row2['ITERATOR'])
        
        print('......df_pred_data: ' + str(df_pred_data.shape))
        
        #do prediction
        df_pred_x = df_pred_data.iloc[:,1:29]
        
        print('......executing predition')  
    
        df_pred_home_expGoals = model_home.predict(df_pred_x)
        df_pred_away_expGoals = model_away.predict(df_pred_x)
    
        #create result data frame
        df_result_data = pd.DataFrame(df_pred_data.iloc[:,0:1])
        df_result_data['feature_type'] = 'random forest/full/' + str(row['FEATURE_TYPE'])
        df_result_data['home_exp_goals'] = df_pred_home_expGoals
        df_result_data['away_exp_goals'] = df_pred_away_expGoals
        df_result_data['interator_param'] = str(row2['ITERATOR'])


        print(df_result_data.head())
        
        print('......data gets written to the db')    
        dwh_conn.import_from_pandas(df_result_data,('BETTING_DV','football_match_his_l_s_ml_poisson_expected_goals'))
    

Loop for feature type: EMA15 COMB
...getting data monthly loops for simulation
...simulation for month 201608
.....getting training data
......training models
......train_x: (3590, 28)
......train home y: (3590,)
......train away y: (3590,)
......getting prediction data
......df_pred_data: (100, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID                   feature_type  \
0  df74d6019d8ec8969da3b8a2d316d111  random forest/full/EMA15 COMB   
1  733d356d163a75c04365593496d232e1  random forest/full/EMA15 COMB   
2  acee56b53a1a434affc5cf8caedd2ef1  random forest/full/EMA15 COMB   
3  6b2f710e0f9b6817cee287a160cfa86f  random forest/full/EMA15 COMB   
4  fe382c3ac79fce6472a8d35927482d7f  random forest/full/EMA15 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.255002        1.018636          201608  
1        1.015276        1.395684          201608  
2        1.263607        1.188050          201608  
3        1.499491        1.227196          2

...simulation for month 201704
.....getting training data
......training models
......train_x: (4960, 28)
......train home y: (4960,)
......train away y: (4960,)
......getting prediction data
......df_pred_data: (280, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID                   feature_type  \
0  45d9adc1ac85ebbc3ae282b4d0dea723  random forest/full/EMA15 COMB   
1  8325fa391bcd8753c483e4b0e36c566f  random forest/full/EMA15 COMB   
2  2b1217948b104826ac1ae44fc92f835c  random forest/full/EMA15 COMB   
3  904cd484e4b3b00ec6b596b70c30a8ee  random forest/full/EMA15 COMB   
4  295f4ef62dfa39d56fe2ed7db954fc51  random forest/full/EMA15 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.810693        1.153646          201704  
1        1.266307        1.289458          201704  
2        1.222921        1.000622          201704  
3        2.036489        0.906450          201704  
4        1.795595        0.980110          201704  
......data gets wri

......getting prediction data
......df_pred_data: (192, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID                   feature_type  \
0  5e9917fda3df324fedf6a04295e30c2d  random forest/full/EMA15 COMB   
1  5eea0a2762973e4f5f685b5925f4ca98  random forest/full/EMA15 COMB   
2  294ab534a68454aa256fbc49f7438136  random forest/full/EMA15 COMB   
3  9e0520b3c9596568753131d5a1b7daf7  random forest/full/EMA15 COMB   
4  e057d17ddda648f1a4ebab74653804c3  random forest/full/EMA15 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.122117        1.546216          201802  
1        1.564164        1.117132          201802  
2        1.191088        1.037736          201802  
3        1.879737        0.808625          201802  
4        1.227766        1.699337          201802  
......data gets written to the db
...simulation for month 201803
.....getting training data
......training models
......train_x: (6681, 28)
......train home y: (6681,)
......train a

...simulation for month 201901
.....getting training data
......training models
......train_x: (8109, 28)
......train home y: (8109,)
......train away y: (8109,)
......getting prediction data
......df_pred_data: (157, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID                   feature_type  \
0  ad8cffffe51814574e20f0b8308c0691  random forest/full/EMA15 COMB   
1  7770852de657253d92f14700bf99af21  random forest/full/EMA15 COMB   
2  cb0357f7edbdf6106e257a63e08b3fee  random forest/full/EMA15 COMB   
3  6059d6f1a7c5b7545cf1179d74a7ddf6  random forest/full/EMA15 COMB   
4  e4e29e114c404e4c2335b29b40a04411  random forest/full/EMA15 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.438037        0.845507          201901  
1        1.267628        1.063514          201901  
2        1.111750        1.556149          201901  
3        1.206122        1.062080          201901  
4        1.695962        1.220205          201901  
......data gets wri

......getting prediction data
......df_pred_data: (173, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID                   feature_type  \
0  fa979e3ef9815e26008c10cb88eb9b7e  random forest/full/EMA15 COMB   
1  a087e1cb1f27050533e588c4d0fe86cc  random forest/full/EMA15 COMB   
2  c737f7880ceb6182c1e56ffb1feb7124  random forest/full/EMA15 COMB   
3  80764e427ec141bd535c48a8c2d9b0bd  random forest/full/EMA15 COMB   
4  85fd785d5dc18573acbeee27e48bca6e  random forest/full/EMA15 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.544608        1.115142          201911  
1        1.245046        2.211535          201911  
2        1.563240        1.079991          201911  
3        0.931438        1.471161          201911  
4        1.142475        2.088252          201911  
......data gets written to the db
...simulation for month 201912
.....getting training data
......training models
......train_x: (9708, 28)
......train home y: (9708,)
......train a

...

...simulation for month 201808
.....getting training data
......training models
......train_x: (7226, 28)
......train home y: (7226,)
......train away y: (7226,)
......getting prediction data
......df_pred_data: (105, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID                   feature_type  \
0  1d18d61987a5d9fcd469b99a367425fa  random forest/full/EMA20 COMB   
1  512a3ed97444ce3e4a6cdafb1cc02229  random forest/full/EMA20 COMB   
2  2cfb22741cbb46ca206c3f0c1f17f64c  random forest/full/EMA20 COMB   
3  5c07b3d64819214e28855b4147cb8e4a  random forest/full/EMA20 COMB   
4  d77559f7adcf804b43cf2bd93f2b4ca8  random forest/full/EMA20 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.440431        1.134206          201808  
1        1.470093        0.989595          201808  
2        1.412021        1.001698          201808  
3        1.359853        1.196049          201808  
4        2.577868        0.579927          201808  
......data gets wri

......getting prediction data
......df_pred_data: (238, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID                   feature_type  \
0  fac3a779b1fb8fadf7731af4f59ea0fb  random forest/full/EMA20 COMB   
1  7a94f38f69d5b18ce933d17b93935f35  random forest/full/EMA20 COMB   
2  8544df7977ba890aff78abf35a7c9e1b  random forest/full/EMA20 COMB   
3  1ca759c47d31938ae0d1a794938ce6e1  random forest/full/EMA20 COMB   
4  2224a062d4127e0c6604501815a63d3a  random forest/full/EMA20 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.098924        1.739165          201904  
1        2.591079        0.948716          201904  
2        1.163403        2.202205          201904  
3        2.356395        0.851692          201904  
4        1.700989        0.826323          201904  
......data gets written to the db
...simulation for month 201905
.....getting training data
......training models
......train_x: (8884, 28)
......train home y: (8884,)
......train a

...simulation for month 202003
.....getting training data
......training models
......train_x: (10293, 28)
......train home y: (10293,)
......train away y: (10293,)
......getting prediction data
......df_pred_data: (60, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID                   feature_type  \
0  98da06a8dcb879de42d7df0ddac017f8  random forest/full/EMA20 COMB   
1  7dfb49ae68215726f5e4cefca04abf59  random forest/full/EMA20 COMB   
2  2e4a8f62723e5cdec91e35a3d61bfdd8  random forest/full/EMA20 COMB   
3  f12efde5297ebf3f2d4aa63b4e54085f  random forest/full/EMA20 COMB   
4  e5e88e433e207e605bb186acabd99386  random forest/full/EMA20 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.349958        1.134712          202003  
1        2.955922        0.697905          202003  
2        1.382812        1.022598          202003  
3        1.634383        1.345553          202003  
4        1.455318        1.328911          202003  
......data gets w

......training models
......train_x: (11245, 28)
......train home y: (11245,)
......train away y: (11245,)
......getting prediction data
......df_pred_data: (244, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID                   feature_type  \
0  0741c4cc4c27a235607ec5143b4a5356  random forest/full/EMA20 COMB   
1  a79ef467383df1d0f6b4acef12f46f5a  random forest/full/EMA20 COMB   
2  746da71c0b33775c166ae2f9a4eee1ba  random forest/full/EMA20 COMB   
3  e4d372d7bd1712a4e8f13320e98d6c29  random forest/full/EMA20 COMB   
4  4815051f51b1ea5314bae55ce317320b  random forest/full/EMA20 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        1.129127        1.393449          202012  
1        1.618499        0.887918          202012  
2        1.850270        0.593464          202012  
3        2.020867        1.292515          202012  
4        1.502206        1.374771          202012  
......data gets written to the db
...simulation for month 202101
.....gett

......getting prediction data
......df_pred_data: (211, 29)
......executing predition
             FOOTBALL_MATCH_HIS_LID                   feature_type  \
0  a808c3a4540d5f91776402344ebfb8a4  random forest/full/EMA20 COMB   
1  079def73a5cc60b5a3500ea4c7116c63  random forest/full/EMA20 COMB   
2  73b27ee9e7b182beb98607ece12f14b8  random forest/full/EMA20 COMB   
3  768135833d8e52cd175afb68c4f5bf21  random forest/full/EMA20 COMB   
4  d0b610ed82674b5a0f3022d1a63fa596  random forest/full/EMA20 COMB   

   home_exp_goals  away_exp_goals interator_param  
0        2.244800        0.809824          202110  
1        1.544398        1.191053          202110  
2        1.228025        1.669017          202110  
3        1.394140        1.075892          202110  
4        2.683698        1.054060          202110  
......data gets written to the db
...simulation for month 202111
.....getting training data
......training models
......train_x: (13108, 28)
......train home y: (13108,)
......train